# Vistelius text digitalization

In [325]:
import re
import numpy as np
import pandas as pd
from collections import Counter

In [357]:
text_match_corr = open("../_CLEANED/text_data_regexed_clean.txt", "r", encoding="utf8")

In [14]:
text_match_corr_list = []

for line in text_match_corr:
    text_match_corr_list.append(line)

In [15]:
text_match_corr_list[:10]

['1- (44°31\'30",138 37\'30")|.Granite alaskitic|.K2|(V.N.Musin,1970). \n',
 '2- (54°12\'10" ,119°24\')|.Granite leucocratic|.Tr|.V.I.Zhigalova|(E.A.Ivanov,1969). \n',
 "3- (62°36',155°36')|.Granite leucocratic coarse-grained|.K2'|.Left-Omsukchan massif|.P.M.Bosek|(O.S.Gracheva,1948). \n",
 "4- (61°35',146°02')|.Granite leucocratic|.K1|.Buksandzhin massif|.A.Kh.Brovtman|(A.F.Mikhaylov,1948). \n",
 "5- (68°55',164°24')|.Granite-porphyry micropegmatitic|.K1 Attykveem massif|.L.G.Semenova|.0th. :S-0.16|(A|.I.Sadovsky|,1963). \n",
 "6- (50°42',113°42')|.Bt granite porphyraceous medium-coarse-grained|.J3|.Sakhanay massif O.S.Beloborod'ko|(N.N.Chaban,1966). \n",
 "7- (47°30' ,136°27.')|.Bt granite|.K2|.N.A.Krivitskaya|(A.A.Golovneva,1950). \n",
 "8- (48°14|~ ,137°28')|.Granite fine-grained|.K2|(V.A.Yarmolyuk,1949).  \n",
 "9- (50°16',112°59')|.Granite leucocratic fine-grained|.J3|.Ongotson stock|.M.I.Romanova|(N.K.Dmitrochenko,1968). \n",
 "10- (51°03',113°33')|.Granite leucocratic fine-grai

In [16]:
regex = r"""
	^[\-\.·,:\s0\']*
	([0-9GSO~iIlJT\)&\'\|]+)  # Group 1 - SampleID
	[\-\s\(\.\{L\\'\'~0·\|]*
	([0-9iIl!pS~o\-•Z\.,&\|]{2,})  # Group 2 - Lattitude degrees
	[°\s90Qg\|]*
	([0-9iIl!pS~roJ\()\.\|]{2,})  # Group 3 - Lattitude minutes
	[\'\\ji\s1;\|]*
	([0-9iIl!pS~norJf\|]{2,})?  # Group 4 - Lattitude seconds (optional)
	[\"\\'\s,~\.;\)\|]*
	([0-9iIl!pS~of\s\$sa·t£\|]{1,})  # Group 5 - Longitude degrees
	[°\s90Qg\|]*
	([0-9iIl!pS~\soJ\-Zj\|]{2,})  # Group 6 - Longitude minutes
	[\'\\\s\|]*
	([0-9iIl!pS~oq]{2,})?  # Group 7 - Longitude seconds (optional)
	([Ww]?)?  # Group 8 - Longitude direction (W)
	[\"\)\}\.,wWt\s\'j\|:]+
	([A-z\s\-~\(\)·0\|\?\']{3,})  # Group 9 - Rock name
	[\.\|,]{2,}
	([A-z0-9\s\-~\']+)  # Group 10 - Whole rock age
	[\.\|\s,\']+
	([A-z0-9\s\-~\|\'\?!]{2,})?  # Group 11 - Massif name (optional)
	(?(11)[\.\|\s,]{2,}|[\.\|\s,]*)
	([A-z0-9\s\.,\'!\|~]{4,})?  # Group 12 - Analyst name (optional)
	(?(12)[\.\|\s,]+|[\.\|\s,]*)
	([A-z0-9\s:\.,~\-+\|\?\']+)?  # Group 13 - Interpretation of "oth." and "det." (optional)
	(?(13)[\.\|\s,]+|[\.\|\s,]*)
	[\(\{]+
	([A-z\.,0-9\s\|\'~\-!\?]+)  # Group 14 - Author and year of the original report
	[\)\}\.\s]+
	"""

In [47]:
# Regular expression for finding sampleID and coordinates
regex_sample_info = re.compile( r"^[\-\.·,:\s0\']*" +
                                r"([0-9GSO~iIlJT\)&\'\|]+)" +  # Group 1 - SampleID
                                r"[\-\s\(\.\{L\\'\'~0·\|]*" +
                                r"([0-9iIl!pS~o\-•Z\.,&\|]{2,})" +  # Group 2 - Lattitude degrees
                                r"[°\s90Qg\|]*" +
                                r"([0-9iIl!pS~roJ\()\.\|]{2,})" +  # Group 3 - Lattitude minutes
                                r"[\'\\ji\s1;\|]*" +
                                r"([0-9iIl!pS~norJf\|]{2,})?" +  # Group 4 - Lattitude seconds (optional)
                                r"[\"\\'\s,~\.;\)\|]*" +
                                r"([0-9iIl!pS~of\s\$sa·t£\|]{1,})" +  # Group 5 - Longitude degrees
                                r"[°\s90Qg\|]*" +
                                r"([0-9iIl!pS~\soJ\-Zj\|]{2,})" +  # Group 6 - Longitude minutes
                                r"[\'\\\s\|]*" +
                                r"([0-9iIl!pS~oq]{2,})?" +  # Group 7 - Longitude seconds (optional)
                                r"([Ww]?)?" +  # Group 8 - Longitude direction (W)
                                r"[\"\)\}\.,wWt\s\'j\|:]+" +
                                r"([A-z\s\-~\(\)·0\|\?\']{3,})" +  # Group 9 - Rock name
                                r"[\.\|,]{2,}" +
                                r"([A-z0-9\s\-~\']+)" +  # Group 10 - Whole rock age
                                r"[\.\|\s,\']+" +
                                r"([A-z0-9\s\-~\|\'\?!]{2,})?" +  # Group 11 - Massif name (optional)
                                r"(?(11)[\.\|\s,]{2,}|[\.\|\s,]*)" +
                                r"([A-z0-9\s\.,\'!\|~]{4,})?" +  # Group 12 - Analyst name (optional)
                                r"(?(12)[\.\|\s,]+|[\.\|\s,]*)" +
                                r"([A-z0-9\s:\.,~\-+\|\?\']+)?" +  # Group 13 - Interpretation of "oth." and "det." (optional)
                                r"(?(13)[\.\|\s,]+|[\.\|\s,]*)" +
                                r"[\(\{]+" +
                                r"([A-z\.,0-9\s\|\'~\-!\?]+)" +  # Group 14 - Author and year of the original report
                                r"[\)\}\.\s]+"

SyntaxError: unexpected character after line continuation character (<ipython-input-47-f3cf472e14d2>, line 3)

In [38]:
# Check if all samples are picked up by sampleID and coordinates regex
for line in text_match_corr:
    print(regex_sample_info.findall(line))

____

In [358]:
matches = re.findall(regex, text_match_corr.read(), re.MULTILINE | re.VERBOSE)

In [359]:
matches_array = np.array(matches)
matches_groups_dict = {}

for groupNum in range(len(matches[0])):
    matches_groups_dict[groupNum] = matches_array[:, groupNum]

___

In [92]:
def try_conversion_to_float(array):
    for i, item in enumerate(array, start=1):
        if len(item) == 0:
            continue
        try:
            float(item)
        except:
            print(i, item)

In [72]:
def replace_characters(array, old, new):
    new_list = []
    
    for item in array:
        new_list.append(item.replace(old, new))
    
    return np.array(new_list)

In [118]:
def check_string_length(array, threshold):
    for i, item in enumerate(array, start=1):
        if len(item) > threshold:
            print(i, item)

## Cleaning columns

### Sample ID's

In [360]:
matches_groups_dict[0] = list(range(1, 4660))

### Coordinates

Expected properties:
 - Integers ranging from
     - 0-90 for lattitude degrees --> No more than 2 numbers
     - 0-180 for longitude degrees --> No more than 3 numbers
     - 0-60 for minutes and seconds --> No more than 2 numbers

In [361]:
matches_groups_dict[1] = replace_characters(matches_groups_dict[1], "I", "1")
matches_groups_dict[1] = replace_characters(matches_groups_dict[1], "|", "")
matches_groups_dict[1] = replace_characters(matches_groups_dict[1], "~", "")
matches_groups_dict[1] = replace_characters(matches_groups_dict[1], ",", "")

In [362]:
try_conversion_to_float(matches_groups_dict[1])

997 &2
1632 6-0
1637 66-0


In [363]:
check_string_length(matches_groups_dict[1], 2)

357 530
656 540
1632 6-0
1637 66-0
2720 136


____

In [364]:
matches_groups_dict[2] = replace_characters(matches_groups_dict[2], "I", "1")
matches_groups_dict[2] = replace_characters(matches_groups_dict[2], "l", "1")
matches_groups_dict[2] = replace_characters(matches_groups_dict[2], "|", "")

In [365]:
try_conversion_to_float(matches_groups_dict[2])

8 14~
14 1i20
1435 1J
1445 16i40
1454 3~
2118 4i
2120 (2
2310 ~3
3127 5r57
3231 ~6
3891 r~
4195 ~4


In [366]:
check_string_length(matches_groups_dict[2], 2)

8 14~
14 1i20
1445 16i40
2437 13.
3127 5r57
4355 101


____

In [367]:
matches_groups_dict[3] = replace_characters(matches_groups_dict[3], "I", "1")
matches_groups_dict[3] = replace_characters(matches_groups_dict[3], "l", "1")

In [368]:
try_conversion_to_float(matches_groups_dict[3])

47 48~
823 ~10
1315 3ri
1459 ~15
1480 40i
1638 50f1
2041 45|
2447 30~
2546 ~16
2716 ~0
3776 2n
4027 ~11
4353 48~
4520 0~
4535 0|


In [369]:
check_string_length(matches_groups_dict[3], 2)

47 48~
823 ~10
1315 3ri
1459 ~15
1480 40i
1638 50f1
2041 45|
2447 30~
2546 ~16
4027 ~11
4353 48~


____

In [370]:
matches_groups_dict[4] = replace_characters(matches_groups_dict[4], "I", "1")
matches_groups_dict[4] = replace_characters(matches_groups_dict[4], "i", "1")
matches_groups_dict[4] = replace_characters(matches_groups_dict[4], "l", "1")
matches_groups_dict[4] = replace_characters(matches_groups_dict[4], "t", "1")
matches_groups_dict[4] = replace_characters(matches_groups_dict[4], " ", "")

In [371]:
try_conversion_to_float(matches_groups_dict[4])

48 14p0
519 f11
638 10$0
916 1~7
982 13~0
1478 11|~0
2080 1so0
2084 17a0
2118 1~2
2901 135~
2954 11£0
3194 117~
3295 138~
3549 14o0
3891 !39
4184 1~8


In [372]:
check_string_length(matches_groups_dict[4], 3)

48 14p0
357 1150
638 10$0
870 1410
982 13~0
1236 1133
1478 11|~0
2080 1so0
2084 17a0
2430 1710
2438 1110
2901 135~
2954 11£0
2981 1109
3125 1310
3194 117~
3295 138~
3549 14o0
4535 7157


____

In [373]:
matches_groups_dict[5] = replace_characters(matches_groups_dict[5], "I", "1")
matches_groups_dict[5] = replace_characters(matches_groups_dict[5], "i", "1")
matches_groups_dict[5] = replace_characters(matches_groups_dict[5], "l", "1")
matches_groups_dict[5] = replace_characters(matches_groups_dict[5], "|", "")
matches_groups_dict[5] = replace_characters(matches_groups_dict[5], "S", "5")
matches_groups_dict[5] = replace_characters(matches_groups_dict[5], " ", "")
matches_groups_dict[5] = replace_characters(matches_groups_dict[5], "o", "0")

In [374]:
try_conversion_to_float(matches_groups_dict[5])

182 0!
825 0J
855 ~0
1797 ~6
2114 0j
2434 2-9
2697 25~
2914 Z1


In [375]:
check_string_length(matches_groups_dict[5], 2)

2434 2-9
2697 25~
3039 -025


____

In [376]:
matches_groups_dict[6] = replace_characters(matches_groups_dict[6], "I", "1")
matches_groups_dict[6] = replace_characters(matches_groups_dict[6], "i", "1")
matches_groups_dict[6] = replace_characters(matches_groups_dict[6], "l", "1")
matches_groups_dict[6] = replace_characters(matches_groups_dict[6], "q", "0")
matches_groups_dict[6] = replace_characters(matches_groups_dict[6], "o", "0")
matches_groups_dict[6] = replace_characters(matches_groups_dict[6], "~", "")

In [377]:
try_conversion_to_float(matches_groups_dict[6])

In [378]:
check_string_length(matches_groups_dict[6], 2)

____

In [402]:
# Move massif names in Age column to Massif column
for i, item in enumerate(matches_groups_dict[9], start=0):
    match = re.findall(r"(o\s)?(\D*\smassif)", item)
    if len(match) != 0:
#         print(match[0][1].strip())
        matches_groups_dict[10][i] += match[0][1].strip()
        matches_groups_dict[9][i] = re.sub(match[0][1], "", item)

    match2 = re.findall(r"(o\s)?(\D*\sstock)", item)
    if len(match2) !=0:
#         print(match2[0][2].strip())
        matches_groups_dict[10][i] += match2[0][1].strip()
        matches_groups_dict[9][i] = re.sub(match2[0][1], "", item)

pass;

In [430]:
# Move massif names in Analyst column to Massif column
for i, item in enumerate(matches_groups_dict[11], start=0):
    match = re.findall(r"^.*massif|^.*mass if|^.*mas sif|^.*mass!f|^.*stock", item)
    if len(match) != 0:
        matches_groups_dict[10][i] += match[0]
        
        matches_groups_dict[11][i] = re.sub(match[0], "", item)
pass;

In [413]:
# Move 'Det' or 'Oth' in Analyst column to Trace Elements column
for i, item in enumerate(matches_groups_dict[11], start=0):
    match = re.findall(r"det$|Det$|^Det|Det.|Det|Oth|Oth$|0th$|^Oth|0th.", item)
    if len(match) != 0:
        matches_groups_dict[12][i] = match[0] + matches_groups_dict[12][i]
        
        matches_groups_dict[11][i] = re.sub(match[0], "", item)
pass;

In [421]:
# Move 'Det' or 'Oth' in Massif column to Trace Elements column
for i, item in enumerate(matches_groups_dict[10], start=0):
    match = re.findall(r"det$|Det$|^Det|Det.|Det|Oth|Oth$|0th$|^Oth|0th.", item)
    if len(match) != 0:
        matches_groups_dict[12][i] = match[0] + matches_groups_dict[12][i]
        
        matches_groups_dict[10][i] = re.sub(match[0], "", item)
pass;

In [431]:
# Move massif names in Trace elements column to Massif column
for i, item in enumerate(matches_groups_dict[12], start=0):
    match = re.findall(r"^.*massif|^.*mass if|^.*mas sif|^.*mass!f|^.*stock", item)
    if len(match) != 0:
        matches_groups_dict[10][i] += match[0]
        
        matches_groups_dict[12][i] = re.sub(match[0], "", item)
pass;

In [406]:
for i, item in enumerate(matches_groups_dict[13], start=0):
    match = re.findall(r"^.*massif|^.*stock", item)
    if len(match) != 0:
        matches_groups_dict[10][i] += match[0]
        
        matches_groups_dict[13][i] = re.sub(match[0], "", item)

In [427]:
# Delete temporary separators '|'
for i in range(8, 14):
    if i == 9:
        matches_groups_dict[i] = replace_characters(matches_groups_dict[i], " ", "")
    if i == 10:
        matches_groups_dict[i] = replace_characters(matches_groups_dict[i], "mas sif", "massif")
        matches_groups_dict[i] = replace_characters(matches_groups_dict[i], "mass if", "massif")
        matches_groups_dict[i] = replace_characters(matches_groups_dict[i], "mass!f", "massif")
        
    if i == 11:
        matches_groups_dict[i] = replace_characters(matches_groups_dict[i], "|.", "")
        
    matches_groups_dict[i] = replace_characters(matches_groups_dict[i], "|", "")

In [433]:
Counter(matches_groups_dict[10]).most_common()

[('', 1956),
 ('Ulakhan-Sis massif', 48),
 ('Khoboyotuu-Echiy massif', 47),
 ("Zimov'e massif", 31),
 ('Bom-Gorkhon massif', 31),
 ('Vladimirsky massif', 29),
 ("Ol'gin massif", 22),
 ('Khatynnakh massif', 20),
 ('Mutukhin Oprichnin massif', 18),
 ('Left-Omsukchan massif', 17),
 ('Chibagalakh massif', 17),
 ('Middle-Khalyin massif', 17),
 ('Pevek massif', 17),
 ("Iul'tin massif", 16),
 ('Taureran massif', 16),
 ('East-Polousny massif', 16),
 ('Valentinovsky massif', 16),
 ('Olenguy massif', 15),
 ('Kontalak massif', 15),
 ('Karakty massif', 15),
 ('Telekay massif', 15),
 ('Taryn massif', 15),
 ('Kontalakan-Marektin massif', 14),
 ('Asakan-Shumilovo massif', 14),
 ('Bakyn massif', 13),
 ('Velitkenay massif', 13),
 ('Khalzan massif', 13),
 ('Gulanin massif', 13),
 ('Samargin massif', 13),
 ('Omchikandin massif', 12),
 ('Zangan massif', 12),
 ('Dintsukhin massif', 12),
 ('Saken massif', 12),
 ("Lis'in massif", 12),
 ('Dybin massif', 12),
 ('Sinanchin massif', 11),
 ('Northern massif', 11)

In [432]:
df_temp = pd.DataFrame.from_dict(matches_groups_dict)
df_temp.to_excel("../_NEEDS_CLEANING/text_matches.xlsx", )

In [418]:
df_temp

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,44,31,30,138,37,30,,Granite alaskitic,K2,,,,"V.N.Musin,1970"
1,2,54,12,10,119,24,,,Granite leucocratic,Tr,,V.I.Zhigalova,,"E.A.Ivanov,1969"
2,3,62,36,,155,36,,,Granite leucocratic coarse-grained,K2',Left-Omsukchan massif,P.M.Bosek,,"O.S.Gracheva,1948"
3,4,61,35,,146,02,,,Granite leucocratic,K1,Buksandzhin massif,A.Kh.Brovtman,,"A.F.Mikhaylov,1948"
4,5,68,55,,164,24,,,Granite-porphyry micropegmatitic,K1,Attykveem massif,L.G.Semenova,0th.:S-0.16,"A.I.Sadovsky,1963"
5,6,50,42,,113,42,,,Bt granite porphyraceous medium-coarse-grained,J3,Sakhanay massif,O.S.Beloborod'ko,,"N.N.Chaban,1966"
6,7,47,30,,136,27,,,Bt granite,K2,,N.A.Krivitskaya,,"A.A.Golovneva,1950"
7,8,48,14~,,137,28,,,Granite fine-grained,K2,,,,"V.A.Yarmolyuk,1949"
8,9,50,16,,112,59,,,Granite leucocratic fine-grained,J3,Ongotson stock,M.I.Romanova,,"N.K.Dmitrochenko,1968"
9,10,51,03,,113,33,,,Granite leucocratic fine-grained,J3,Kalangin massif,N.A.Ogneva,,"E.V.Barabashev,1962"
